# 📊 **Exploratory Data Analysis (EDA)**

This notebook is designed to explore the cleaned *delinquency risk* dataset and gain a deeper understanding of its structure and characteristics.

## 🎯 **Objectives of EDA**

- Understand the overall structure of the dataset (rows, columns, data types)  
- Identify missing values, inconsistencies, and potential data quality issues  
- Analyze the distributions of numerical and categorical features  
- Detect patterns, anomalies, and potential outliers  
- Generate insights that can support future modeling or feature engineering steps  

In this stage, **no modifications are made to the data** — the dataset is only analyzed.

## 📚 **1. Loading Required Libraries**

In the following cell, essential libraries such as pandas are imported, and the dataset is loaded for initial exploration.  
Basic structural and statistical summaries are also displayed.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df_clean = pd.read_csv("delinquency_risk_cleaned.csv")

print("SHAPE:", df_clean.shape)

print("\nINFO\n")
df_clean.info()

print("\nDESCRIBE NUMERIC\n")
display(df_clean.describe())

print("\nDESCRIBE CATEGORICAL\n")
display(df_clean.describe(include='object'))

print("\nHEAD\n")
display(df_clean.head(10))


## 🔢 **2. Identifying Numerical and Categorical Columns**

In this section, we separate numerical and categorical features to structure the exploratory analysis. This helps ensure that each variable type is analyzed using appropriate statistical and visualization techniques.

### ✔ Steps Performed
- Create a working copy of the cleaned dataset  
- Identify all numerical columns using `select_dtypes`  
- Exclude technical identifiers such as **id** from analysis  
- Identify categorical columns  
- Generate a descriptive summary table for numerical features  
- Calculate the **zero ratio**, showing the proportion of zero values in each numerical column

These steps provide a clear overview of the dataset's feature types and allow deeper analysis in later sections.


In [ ]:
df_eda = df_clean.copy()

num_cols = df_eda.select_dtypes(include=[np.number]).columns
num_cols = [c for c in num_cols if c.lower() != "id"]

cat_cols = df_eda.select_dtypes(include=["object", "category"]).columns

print("Sayısal kolonlar:", num_cols)
print("Kategorik kolonlar:", list(cat_cols))

num_summary = df_eda[num_cols].describe().T
num_summary["zero_ratio"] = (df_eda[num_cols] == 0).mean()
num_summary[["count","mean","std","min","25%","50%","75%","max","zero_ratio"]]


## 📈 **3. Visualizing Numerical Feature Distributions**

To better understand the behavior of numerical variables, this section visualizes each numerical column using both a histogram (with KDE) and a boxplot.  
These visualizations help identify:

- The distribution shape (normal, skewed, multimodal)  
- Presence of outliers  
- Potential data quality issues  
- Range, spread, and central tendencies  

### ✔ How the function works
- A custom function `plot_numeric_distribution()` is defined  
- For each numerical column, the function draws:  
  - **Histogram + KDE curve**  
  - **Boxplot** to highlight outliers  
- The function is applied to all numerical features in `num_cols`

In [ ]:
def plot_numeric_distribution(df, col):
    fig, axes = plt.subplots(1, 2, figsize=(10, 4))

    sns.histplot(df[col].dropna(), kde=True, ax=axes[0])
    axes[0].set_title(f"{col} - Histogram/KDE")

    sns.boxplot(x=df[col], ax=axes[1])
    axes[1].set_title(f"{col} - Boxplot")

    plt.tight_layout()
    plt.show()

for col in num_cols:
    plot_numeric_distribution(df_eda, col)


## 🧩 **4. Exploring Categorical Feature Distributions**

This section focuses on examining the distribution of categorical variables. Understanding the frequency of category values helps identify dominant groups, rare categories, and potential inconsistencies.

### ✔ Steps Performed
- For each categorical column, the top 10 most frequent categories are displayed  
- A bar chart is generated to visualize the frequency distribution  
- Category labels are rotated for readability  
- This process helps detect data imbalance, uncommon labels, or potential issues before modeling

### Code Logic (plain text)
for each column in cat_cols:  
• print its top 10 value counts  
• plot a bar chart of the top 10 categories  
• display the plot with appropriate title and formatting


In [ ]:
for col in cat_cols:
    print(f"\n{col} value_counts:")
    print(df_eda[col].value_counts().head(10))

    plt.figure(figsize=(6,4))
    df_eda[col].value_counts().head(10).plot(kind="bar")
    plt.title(f"Top 10 categories - {col}")
    plt.xticks(rotation=45)
    plt.show()


## 🔗 **5. Correlation Analysis of Numerical Features**

In this section, a correlation matrix is computed to examine relationships between numerical variables. Correlation analysis helps identify potential multicollinearity, strong linear relationships, or variables that may contribute similarly to predictive models.

### ✔ Steps Performed
- Select all numerical features except the identifier column  
- Compute the pairwise correlation matrix  
- Visualize the matrix using a heatmap with a diverging color palette  
- Highlight positive and negative correlations centered around zero  

### Code Logic (plain text)
• extract numerical columns  
• remove the “id” field from analysis  
• compute correlations using df_eda[num_cols].corr()  
• display a heatmap (coolwarm, center=0) for clearer interpretation


In [ ]:
#korelasyon matrisi
num_cols = df_eda.select_dtypes(include=[np.number]).columns
num_cols = [c for c in num_cols if c.lower() != "id"]

corr = df_eda[num_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr, cmap="coolwarm", center=0)
plt.title("Correlation Matrix (Numeric Features)")
plt.show()


## 📉 **6. Detailed Distribution Plots for Numerical Features**

This section provides a more detailed visualization of each numerical variable by displaying both a histogram (with KDE) and a boxplot side by side. These visualizations help deepen the understanding of numerical feature behavior and potential anomalies.

### ✔ Insights Gained
- Histogram + KDE reveals the overall distribution shape  
- Boxplot highlights outliers and the spread of values  
- Combined, these plots offer a clear view of skewness, variance, and extreme values  
- Supports decisions for potential transformations or normalization steps  

### Code Logic (plain text)
for each numerical column:  
• create a two-panel figure  
• left panel → histogram + KDE curve  
• right panel → boxplot for outlier detection  
• adjust layout and display the plots  


In [ ]:
for col in num_cols:
    plt.figure(figsize=(10,4))

    plt.subplot(1,2,1)
    sns.histplot(df_eda[col].dropna(), kde=True)
    plt.title(f"{col} - Histogram/KDE")

    plt.subplot(1,2,2)
    sns.boxplot(x=df_eda[col])
    plt.title(f"{col} - Boxplot")

    plt.tight_layout()
    plt.show()


## 🚦 **7. Creating Risk Segments Based on Time Since Last Delinquency**

In this section, customers are segmented according to the number of months since their last delinquency event. This transformation helps classify customers into meaningful risk groups and supports downstream modeling or reporting.

### ✔ Steps Performed
- Filter out customers whose delinquency value is labeled as "Never"  
- Convert the delinquency field into a numeric format  
- Define risk segments using `pd.cut` based on time intervals  
- Count the number of customers in each risk group  
- Visualize the resulting distribution using a bar chart  

### ✔ Insights Gained
- Customers with recent delinquency (0–6 months) represent the **highest risk**  
- Those with 6–24 months since delinquency fall into a **medium-risk** category  
- Customers with 24+ months show signs of **improvement or recovery**  
- This segmentation allows institutions to tailor risk strategies and customer approaches  


In [ ]:
df_delinq = df_clean[df_clean["mths_since_last_delinq"] != "Never"].copy()

# 2) Convert to numeric
df_delinq["mths_since_last_delinq_num"] = pd.to_numeric(
    df_delinq["mths_since_last_delinq"],
    errors="coerce"
)

# 3) Define risk segments using pd.cut
# 0–6  -> High Risk
# 6–24 -> Medium Risk
# 24+  -> Recovery Signal

bins = [0, 6, 24, np.inf]
labels = ["High Risk 0–6 Months", "Medium Risk 6–24 Months", "Recovery Signal 24+ Months"]

df_delinq["risk_segment"] = pd.cut(
    df_delinq["mths_since_last_delinq_num"],
    bins=bins,
    labels=labels,
    right=False  # 0-6, 6-24, 24+
)

# 4) Count customers by risk segments
risk_counts = df_delinq["risk_segment"].value_counts().reindex(labels)

print(risk_counts)

# 5) Plot bar chart
plt.figure(figsize=(6,4))
sns.barplot(x=risk_counts.index, y=risk_counts.values)
plt.title("Risk Segments Based on Months Since Last Delinquency")
plt.xlabel("Risk Segment")
plt.ylabel("Customer Count")
plt.tight_layout()
plt.show()



## 🧭 **8. Constructing a Comprehensive Behavioral Risk Profile**

In this section, a consolidated **risk_profile** variable is created by combining multiple behavioral indicators such as recent delinquency, charge-off events, credit card delays, and severe past-due history.  
The objective is to classify customers into prioritized risk tiers based on their credit behavior.

### ✔ Steps Performed
- Prepare a numeric version of the `mths_since_recent_revol_delinq` column (convert “Never” → NaN)  
- Define behavioral risk flags:
  - **Mild delay** – presence of 30+ day delinquency  
  - **Credit card–driven delinquency** – recent revolving delinquency (≤ 3 months)  
  - **Charge-off / collections history** – severe financial distress indicators  
  - **Severe past-due accounts** – 120+ day delinquency at any time  
- Combine these indicators into a single prioritized risk profile using `np.select`  
- Compute counts and percentages for each risk segment  
- Visualize the distribution with a bar chart  

### ✔ Insights Gained
- The profile hierarchy emphasizes *severity first*, ensuring customers with charge-offs or severe delinquencies are captured at the top of the risk spectrum  
- Recent revolving delinquency helps detect short-term credit card stress  
- The final profile provides a clean, interpretable segmentation for reporting or modeling  


In [ ]:
# 0) DataFrame to work with
df_seg = df_clean.copy()

# 1) Prepare numeric column for mths_since_recent_revol_delinq (Never -> NaN)
if "mths_since_recent_revol_delinq_num" in df_seg.columns:
    num_col = "mths_since_recent_revol_delinq_num"
else:
    df_seg["mths_since_recent_revol_delinq_num"] = pd.to_numeric(
        df_seg["mths_since_recent_revol_delinq"].replace("Never", np.nan),
        errors="coerce"
    )
    num_col = "mths_since_recent_revol_delinq_num"

# 2) Define flags according to your rules

# Mild delay history: num_tl_30dpd > 0
mild_delay_flag = df_seg["num_tl_30dpd"] > 0

# Credit card–driven delinquency: mths_since_recent_revol_delinq close to 0
# Here, "close to 0" is defined as last 3 months (<= 3). You may adjust to 6 or 12 months if needed.
cc_recent_flag = df_seg[num_col].notna() & (df_seg[num_col] <= 3)

# Charge-off / collections history: chargeoff_within_12_mths > 0 or collections_12_mths_ex_med > 0
chargeoff_flag = (df_seg["chargeoff_within_12_mths"] > 0) | \
                 (df_seg["collections_12_mths_ex_med"] > 0)

# Severe delinquency history: num_accts_ever_120_pd > 0
heavy_delay_flag = df_seg["num_accts_ever_120_pd"] > 0

# 3) Create a single risk_profile column based on priority order
# Priority: Charge-off > Severe Delay > Credit Card Delay > Mild Delay > Clean Profile

conditions = [
    chargeoff_flag,
    heavy_delay_flag,
    cc_recent_flag,
    mild_delay_flag
]

choices = [
    "Charge-off / Collections History",
    "Severe Delinquency History",
    "Credit Card–Driven Delinquency",
    "Mild Delinquency History"
]

df_seg["risk_profile"] = np.select(
    conditions,
    choices,
    default="Clean Profile"
)

# 4) Segment distribution (count and percentage)
order = [
    "Clean Profile",
    "Mild Delinquency History",
    "Credit Card–Driven Delinquency",
    "Severe Delinquency History",
    "Charge-off / Collections History"
]

profile_counts = df_seg["risk_profile"].value_counts().reindex(order)
profile_pct = profile_counts / len(df_seg)

summary = pd.DataFrame({
    "count": profile_counts,
    "ratio": (profile_pct * 100).round(2)
})
print(summary)

# 5) Plot bar chart
plt.figure(figsize=(8,4))
sns.barplot(x=profile_counts.index, y=profile_counts.values)
plt.title("Customer Risk Profile Distribution")
plt.xlabel("Risk Profile")
plt.ylabel("Customer Count")
plt.xticks(rotation=20)
plt.tight_layout()
plt.show()
